# 12.4 记忆聊天机器人

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-org/ai-first-app/blob/main/demos/12-rag-memory/memory_chatbot.ipynb)

**预计 API 费用: ~$0.02**

本 Notebook 实现带记忆的聊天机器人。

In [ ]:
!pip install -q langchain langchain-openai

## 实验 1: 基础对话记忆

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

# 创建记忆
memory = ConversationBufferMemory()

# 创建对话链
conversation = ConversationChain(
    llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=memory,
    verbose=True
)

# 对话测试
print("=== 对话 1 ===")
response1 = conversation.predict(input="我叫小明")
print(f"AI: {response1}")

print("\n=== 对话 2 ===")
response2 = conversation.predict(input="我 25 岁,喜欢编程")
print(f"AI: {response2}")

print("\n=== 对话 3 ===")
response3 = conversation.predict(input="我叫什么名字?多大年纪?")
print(f"AI: {response3}")

# 查看记忆
print("\n=== 记忆内容 ===")
print(memory.buffer)

## 实验 2: 窗口记忆

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# 只保留最近 3 轮对话
memory_window = ConversationBufferWindowMemory(k=3)
conversation_window = ConversationChain(
    llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=memory_window
)

conversations = [
    "我叫小明",
    "我 25 岁",
    "我在北京",
    "我喜欢编程",
    "我的目标是成为 AI 工程师",
    "我叫什么名字?"  # 第 1 轮应该被遗忘了
]

for i, msg in enumerate(conversations):
    print(f"\n=== 轮次 {i+1} ===")
    print(f"用户: {msg}")
    response = conversation_window.predict(input=msg)
    print(f"AI: {response}")

## 实验 3: 总结记忆

In [ ]:
from langchain.memory import ConversationSummaryMemory

# 自动总结历史
memory_summary = ConversationSummaryMemory(llm=ChatOpenAI(model="gpt-4o-mini"))
conversation_summary = ConversationChain(
    llm=ChatOpenAI(model="gpt-4o-mini"),
    memory=memory_summary
)

# 多轮对话
print("=== 多轮对话 ===")
response1 = conversation_summary.predict(input="我叫小明,25 岁,在北京工作")
print(f"AI: {response1}")

response2 = conversation_summary.predict(input="我喜欢编程,最近在学 Python 和 AI")
print(f"AI: {response2}")

response3 = conversation_summary.predict(input="我的目标是成为一名优秀的 AI 工程师")
print(f"AI: {response3}")

# 查看总结
print("\n=== 记忆总结 ===")
print(memory_summary.buffer)

## 实验 4: 自定义记忆管理

In [ ]:
from openai import OpenAI
import json

client = OpenAI()

class CustomMemory:
    def __init__(self, max_turns=10):
        self.history = []
        self.max_turns = max_turns
        self.user_profile = {}
    
    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})
        # 保持最近 N 轮
        if len(self.history) > self.max_turns * 2:
            self.history = self.history[-self.max_turns * 2:]
    
    def extract_user_info(self, text: str):
        # 简单提取(实际应用中用 LLM)
        if "叫" in text and "我" in text:
            # 提取名字
            pass
    
    def get_messages(self):
        system_msg = {
            "role": "system",
            "content": f"你是一个友好的助手。用户信息: {json.dumps(self.user_profile, ensure_ascii=False)}"
        }
        return [system_msg] + self.history

# 使用
memory = CustomMemory(max_turns=5)

def chat(user_input: str):
    memory.add_message("user", user_input)
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=memory.get_messages()
    )
    
    assistant_reply = response.choices[0].message.content
    memory.add_message("assistant", assistant_reply)
    
    return assistant_reply

# 测试
print("=== 自定义记忆聊天 ===")
print("用户: 我叫小明")
print(f"AI: {chat('我叫小明')}")

print("\n用户: 我喜欢编程")
print(f"AI: {chat('我喜欢编程')}")

print("\n用户: 我叫什么名字?")
print(f"AI: {chat('我叫什么名字?')}")

## 实验 5: 向量记忆

In [ ]:
from langchain.memory import VectorStoreRetrieverMemory
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# 创建向量存储
vectorstore = Chroma(
    embedding_function=OpenAIEmbeddings(),
    persist_directory="./memory_db"
)

# 创建向量记忆
memory_vector = VectorStoreRetrieverMemory(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3})
)

# 保存对话
memory_vector.save_context(
    {"input": "我喜欢 Python 编程"},
    {"output": "Python 是一门很棒的语言!"}
)

memory_vector.save_context(
    {"input": "我在学习机器学习"},
    {"output": "机器学习很有前景!"}
)

memory_vector.save_context(
    {"input": "我最近在看深度学习的书"},
    {"output": "深度学习是 AI 的核心技术。"}
)

# 检索相关记忆
print("=== 检索相关记忆 ===")
result = memory_vector.load_memory_variables({"prompt": "我喜欢什么语言?"})
print(result)

## 动手练习

1. **实现用户画像**: 从对话中提取并保存用户信息
2. **持久化存储**: 将记忆保存到 Redis 或 MongoDB
3. **混合记忆**: 结合短期、中期、长期记忆
4. **记忆召回策略**: 根据相关性智能选择记忆

---

## 关键要点

1. **三种短期记忆**: Buffer(全部)、Window(窗口)、Summary(总结)
2. **长期记忆**: 用户画像、向量存储
3. **权衡**: 记忆越多越好?不是!要平衡成本和效果
4. **持久化**: 生产环境需要数据库存储

---

**恭喜!** 完成了 RAG & 记忆存储章节。

**下一步**: 进入 [第 13 章:生产化部署](../../docs/13-production/index.md)